# Пример решения

In [22]:
import pandas
import xarray
import requests
import datetime
import numpy as np
from tqdm import tqdm_notebook as tqdm
import catboost

## NCEP Dataset

Погодные данные из проекта [NCEP Reanalysis 2](https://www.esrl.noaa.gov/psd/data/gridded/data.ncep.reanalysis2.html) — усреднённые за день температура воздуха, относительная влажность и компоненты ветра. Данные можно получить с 1979 года.

Загрузите наборы данных в каталог `data/ncep/`:
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/air.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/uwnd.2018.nc
- https://www.esrl.noaa.gov/psd/thredds/fileServer/Datasets/ncep/rhum.2018.nc

In [9]:
ncep_data = []
year = 2018
for var in ('air', 'uwnd', 'rhum'):
    dataset_filename = 'data/ncep/{}.{}.nc'.format(var, year)
    ncep_data.append(xarray.open_dataset(dataset_filename))
ncep_data = xarray.merge(ncep_data)

C:\Users\Xiaomi Pro\Anaconda3\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'air' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'uwnd' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,
C:\Users\Xiaomi Pro\Anaconda3\lib\site-packages\xarray\conventions.py:494: SerializationWarning: variable 'rhum' has multiple fill values {-99, 32767}, decoding all values to NaN.
  use_cftime=use_cftime,


## Набор признаков на основе данных NCEP

Ищем наиболее близкий к точке узел сетки в наборе NCEP, в качестве признаков значения переменных зарегистрированные в день регистрации точки и агрегированные показатели за период от 1 до 3х недель до момента регистрации точки.

In [10]:
def extract_features(row):
    point = ncep_data.sel(
        lon=row['longitude'],
        lat=row['latitude'],
        level=1000,
        method='nearest',
    )

    p1w = point.rolling(time=7).mean()
    p2w = point.rolling(time=14).mean()
    p3w = point.rolling(time=21).mean()
    
    date = row['date']
    v = point.sel(time=date)
    v1w = p1w.sel(time=date)
    v2w = p2w.sel(time=date)
    v3w = p3w.sel(time=date)
    
    return {
        'fire_id': row['fire_id'],
        'fire_type': row['fire_type'],
        'fire_type_name': row['fire_type_name'],
        'date': row['date'], 
        'temperature': v.air.values.item(0),
        'humidity': v.rhum.values.item(0),
        'uwind': v.uwnd.values.item(0),
        't1w': v1w.air.values.item(0),
        't2w': v2w.air.values.item(0),
        't3w': v3w.air.values.item(0),
        'h1w': v1w.rhum.values.item(0),
        'h2w': v2w.rhum.values.item(0),
        'h3w': v3w.rhum.values.item(0)
    }

## Выборка для обучения

In [12]:
df_train = pandas.read_csv('data/wildfires_train.csv')
df_subsample = df_train.query('(date > "2018") & (date < "2019")').sample(n=2000)

df_features = []
for i, row in tqdm(df_subsample.iterrows(), total=df_subsample.shape[0]):
    features = extract_features(row)
    df_features.append(features)
df_features = pandas.DataFrame(df_features)
df_features.set_index('fire_id', inplace=True)

## Обучение классификатора

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [14]:
X = df_features.iloc[:, 3:].fillna(0)
y = df_features['fire_type']

In [23]:
fire_classifier = catboost.CatBoostClassifier()

In [28]:
cross_val_score(
    fire_classifier, 
    X, y, 
    cv=5, 
)

0:	learn: 2.2750964	total: 42.1ms	remaining: 42s
1:	learn: 2.2457127	total: 68.5ms	remaining: 34.2s
2:	learn: 2.2197661	total: 96.1ms	remaining: 31.9s
3:	learn: 2.1933173	total: 129ms	remaining: 32.1s
4:	learn: 2.1689487	total: 162ms	remaining: 32.3s
5:	learn: 2.1461532	total: 189ms	remaining: 31.3s
6:	learn: 2.1254760	total: 226ms	remaining: 32.1s
7:	learn: 2.1053964	total: 259ms	remaining: 32.1s
8:	learn: 2.0862529	total: 293ms	remaining: 32.2s
9:	learn: 2.0693874	total: 326ms	remaining: 32.3s
10:	learn: 2.0510090	total: 360ms	remaining: 32.3s
11:	learn: 2.0331560	total: 398ms	remaining: 32.8s
12:	learn: 2.0162498	total: 426ms	remaining: 32.4s
13:	learn: 2.0010148	total: 463ms	remaining: 32.6s
14:	learn: 1.9857634	total: 496ms	remaining: 32.6s
15:	learn: 1.9706386	total: 535ms	remaining: 32.9s
16:	learn: 1.9573824	total: 566ms	remaining: 32.7s
17:	learn: 1.9436555	total: 594ms	remaining: 32.4s
18:	learn: 1.9301504	total: 631ms	remaining: 32.6s
19:	learn: 1.9176733	total: 661ms	remain

161:	learn: 1.4032267	total: 4.85s	remaining: 25.1s
162:	learn: 1.4013276	total: 4.87s	remaining: 25s
163:	learn: 1.4003549	total: 4.9s	remaining: 25s
164:	learn: 1.3994793	total: 4.93s	remaining: 24.9s
165:	learn: 1.3979344	total: 4.96s	remaining: 24.9s
166:	learn: 1.3971639	total: 4.98s	remaining: 24.9s
167:	learn: 1.3957991	total: 5.01s	remaining: 24.8s
168:	learn: 1.3946366	total: 5.04s	remaining: 24.8s
169:	learn: 1.3928825	total: 5.08s	remaining: 24.8s
170:	learn: 1.3914699	total: 5.11s	remaining: 24.8s
171:	learn: 1.3905990	total: 5.15s	remaining: 24.8s
172:	learn: 1.3892950	total: 5.18s	remaining: 24.8s
173:	learn: 1.3881301	total: 5.21s	remaining: 24.7s
174:	learn: 1.3869425	total: 5.24s	remaining: 24.7s
175:	learn: 1.3855197	total: 5.26s	remaining: 24.6s
176:	learn: 1.3842766	total: 5.3s	remaining: 24.6s
177:	learn: 1.3832165	total: 5.32s	remaining: 24.6s
178:	learn: 1.3820046	total: 5.36s	remaining: 24.6s
179:	learn: 1.3811672	total: 5.39s	remaining: 24.6s
180:	learn: 1.3798

320:	learn: 1.2414082	total: 9.59s	remaining: 20.3s
321:	learn: 1.2408803	total: 9.62s	remaining: 20.3s
322:	learn: 1.2404514	total: 9.65s	remaining: 20.2s
323:	learn: 1.2395304	total: 9.69s	remaining: 20.2s
324:	learn: 1.2387394	total: 9.72s	remaining: 20.2s
325:	learn: 1.2381089	total: 9.75s	remaining: 20.2s
326:	learn: 1.2377490	total: 9.79s	remaining: 20.1s
327:	learn: 1.2366236	total: 9.81s	remaining: 20.1s
328:	learn: 1.2357724	total: 9.84s	remaining: 20.1s
329:	learn: 1.2347908	total: 9.87s	remaining: 20s
330:	learn: 1.2339865	total: 9.9s	remaining: 20s
331:	learn: 1.2332401	total: 9.94s	remaining: 20s
332:	learn: 1.2324069	total: 9.96s	remaining: 20s
333:	learn: 1.2320211	total: 10s	remaining: 19.9s
334:	learn: 1.2313349	total: 10s	remaining: 19.9s
335:	learn: 1.2304928	total: 10.1s	remaining: 19.9s
336:	learn: 1.2301106	total: 10.1s	remaining: 19.8s
337:	learn: 1.2289150	total: 10.1s	remaining: 19.8s
338:	learn: 1.2282315	total: 10.1s	remaining: 19.8s
339:	learn: 1.2274491	tot

479:	learn: 1.1261334	total: 14.3s	remaining: 15.5s
480:	learn: 1.1255145	total: 14.3s	remaining: 15.5s
481:	learn: 1.1251076	total: 14.4s	remaining: 15.4s
482:	learn: 1.1243915	total: 14.4s	remaining: 15.4s
483:	learn: 1.1239545	total: 14.4s	remaining: 15.4s
484:	learn: 1.1230271	total: 14.4s	remaining: 15.3s
485:	learn: 1.1226038	total: 14.5s	remaining: 15.3s
486:	learn: 1.1217105	total: 14.5s	remaining: 15.3s
487:	learn: 1.1214938	total: 14.5s	remaining: 15.2s
488:	learn: 1.1206751	total: 14.6s	remaining: 15.2s
489:	learn: 1.1201019	total: 14.6s	remaining: 15.2s
490:	learn: 1.1194461	total: 14.6s	remaining: 15.1s
491:	learn: 1.1191719	total: 14.6s	remaining: 15.1s
492:	learn: 1.1188164	total: 14.7s	remaining: 15.1s
493:	learn: 1.1182032	total: 14.7s	remaining: 15s
494:	learn: 1.1169124	total: 14.7s	remaining: 15s
495:	learn: 1.1160464	total: 14.7s	remaining: 15s
496:	learn: 1.1156329	total: 14.8s	remaining: 14.9s
497:	learn: 1.1152165	total: 14.8s	remaining: 14.9s
498:	learn: 1.1144

638:	learn: 1.0252639	total: 18.9s	remaining: 10.7s
639:	learn: 1.0245410	total: 18.9s	remaining: 10.7s
640:	learn: 1.0240757	total: 19s	remaining: 10.6s
641:	learn: 1.0233016	total: 19s	remaining: 10.6s
642:	learn: 1.0225620	total: 19s	remaining: 10.6s
643:	learn: 1.0219818	total: 19.1s	remaining: 10.5s
644:	learn: 1.0212432	total: 19.1s	remaining: 10.5s
645:	learn: 1.0208219	total: 19.1s	remaining: 10.5s
646:	learn: 1.0198118	total: 19.2s	remaining: 10.5s
647:	learn: 1.0193382	total: 19.2s	remaining: 10.4s
648:	learn: 1.0191777	total: 19.2s	remaining: 10.4s
649:	learn: 1.0181480	total: 19.3s	remaining: 10.4s
650:	learn: 1.0175610	total: 19.3s	remaining: 10.3s
651:	learn: 1.0167401	total: 19.3s	remaining: 10.3s
652:	learn: 1.0160749	total: 19.3s	remaining: 10.3s
653:	learn: 1.0157681	total: 19.4s	remaining: 10.2s
654:	learn: 1.0149411	total: 19.4s	remaining: 10.2s
655:	learn: 1.0142131	total: 19.4s	remaining: 10.2s
656:	learn: 1.0138270	total: 19.5s	remaining: 10.2s
657:	learn: 1.0129

...

462:	learn: 1.1248397	total: 14s	remaining: 16.3s
463:	learn: 1.1243355	total: 14.1s	remaining: 16.3s
464:	learn: 1.1235781	total: 14.1s	remaining: 16.3s
465:	learn: 1.1229003	total: 14.2s	remaining: 16.3s
466:	learn: 1.1224218	total: 14.2s	remaining: 16.2s
467:	learn: 1.1219487	total: 14.3s	remaining: 16.2s
468:	learn: 1.1211051	total: 14.3s	remaining: 16.2s
469:	learn: 1.1205453	total: 14.3s	remaining: 16.2s
470:	learn: 1.1199795	total: 14.4s	remaining: 16.1s
471:	learn: 1.1196750	total: 14.4s	remaining: 16.1s
472:	learn: 1.1194118	total: 14.4s	remaining: 16.1s
473:	learn: 1.1185287	total: 14.5s	remaining: 16.1s
474:	learn: 1.1179350	total: 14.5s	remaining: 16s
475:	learn: 1.1168363	total: 14.5s	remaining: 16s
476:	learn: 1.1163617	total: 14.6s	remaining: 16s
477:	learn: 1.1154235	total: 14.6s	remaining: 15.9s
478:	learn: 1.1147962	total: 14.6s	remaining: 15.9s
479:	learn: 1.1140321	total: 14.7s	remaining: 15.9s
480:	learn: 1.1137960	total: 14.7s	remaining: 15.9s
481:	learn: 1.113250

621:	learn: 1.0276520	total: 20.2s	remaining: 12.3s
622:	learn: 1.0275390	total: 20.2s	remaining: 12.2s
623:	learn: 1.0272842	total: 20.2s	remaining: 12.2s
624:	learn: 1.0269991	total: 20.3s	remaining: 12.2s
625:	learn: 1.0268193	total: 20.3s	remaining: 12.1s
626:	learn: 1.0265431	total: 20.4s	remaining: 12.1s
627:	learn: 1.0259805	total: 20.4s	remaining: 12.1s
628:	learn: 1.0258126	total: 20.4s	remaining: 12.1s
629:	learn: 1.0255852	total: 20.5s	remaining: 12s
630:	learn: 1.0253201	total: 20.5s	remaining: 12s
631:	learn: 1.0248772	total: 20.6s	remaining: 12s
632:	learn: 1.0246132	total: 20.6s	remaining: 11.9s
633:	learn: 1.0240291	total: 20.6s	remaining: 11.9s
634:	learn: 1.0234273	total: 20.7s	remaining: 11.9s
635:	learn: 1.0229720	total: 20.7s	remaining: 11.9s
636:	learn: 1.0224846	total: 20.8s	remaining: 11.8s
637:	learn: 1.0221575	total: 20.8s	remaining: 11.8s
638:	learn: 1.0214151	total: 20.8s	remaining: 11.8s
639:	learn: 1.0210944	total: 20.9s	remaining: 11.7s
640:	learn: 1.0204

780:	learn: 0.9499841	total: 26.6s	remaining: 7.45s
781:	learn: 0.9494611	total: 26.6s	remaining: 7.42s
782:	learn: 0.9488446	total: 26.6s	remaining: 7.38s
783:	learn: 0.9486668	total: 26.7s	remaining: 7.35s
784:	learn: 0.9480966	total: 26.7s	remaining: 7.32s
785:	learn: 0.9474943	total: 26.8s	remaining: 7.29s
786:	learn: 0.9471840	total: 26.8s	remaining: 7.25s
787:	learn: 0.9464069	total: 26.8s	remaining: 7.22s
788:	learn: 0.9458794	total: 26.9s	remaining: 7.18s
789:	learn: 0.9454136	total: 26.9s	remaining: 7.15s
790:	learn: 0.9450293	total: 26.9s	remaining: 7.11s
791:	learn: 0.9446111	total: 27s	remaining: 7.08s
792:	learn: 0.9443991	total: 27s	remaining: 7.04s
793:	learn: 0.9439369	total: 27s	remaining: 7.01s
794:	learn: 0.9438525	total: 27.1s	remaining: 6.98s
795:	learn: 0.9436720	total: 27.1s	remaining: 6.94s
796:	learn: 0.9432263	total: 27.1s	remaining: 6.91s
797:	learn: 0.9430273	total: 27.2s	remaining: 6.88s
798:	learn: 0.9425069	total: 27.2s	remaining: 6.84s
799:	learn: 0.9421

939:	learn: 0.8781871	total: 32.4s	remaining: 2.06s
940:	learn: 0.8775383	total: 32.4s	remaining: 2.03s
941:	learn: 0.8773307	total: 32.4s	remaining: 2s
942:	learn: 0.8765170	total: 32.5s	remaining: 1.96s
943:	learn: 0.8758534	total: 32.5s	remaining: 1.93s
944:	learn: 0.8756288	total: 32.5s	remaining: 1.89s
945:	learn: 0.8750612	total: 32.6s	remaining: 1.86s
946:	learn: 0.8748830	total: 32.6s	remaining: 1.82s
947:	learn: 0.8745091	total: 32.6s	remaining: 1.79s
948:	learn: 0.8740484	total: 32.7s	remaining: 1.75s
949:	learn: 0.8736505	total: 32.7s	remaining: 1.72s
950:	learn: 0.8733932	total: 32.7s	remaining: 1.69s
951:	learn: 0.8726727	total: 32.8s	remaining: 1.65s
952:	learn: 0.8722619	total: 32.8s	remaining: 1.62s
953:	learn: 0.8718597	total: 32.8s	remaining: 1.58s
954:	learn: 0.8715109	total: 32.9s	remaining: 1.55s
955:	learn: 0.8705824	total: 32.9s	remaining: 1.51s
956:	learn: 0.8703000	total: 32.9s	remaining: 1.48s
957:	learn: 0.8697688	total: 33s	remaining: 1.45s
958:	learn: 0.869

array([0.50372208, 0.44416873, 0.44887781, 0.47738693, 0.47341772])

In [29]:
fire_classifier.fit(X, y)

0:	learn: 2.2758474	total: 33.4ms	remaining: 33.4s
1:	learn: 2.2500412	total: 72.5ms	remaining: 36.2s
2:	learn: 2.2251043	total: 101ms	remaining: 33.6s
3:	learn: 2.1995124	total: 138ms	remaining: 34.3s
4:	learn: 2.1744386	total: 172ms	remaining: 34.2s
5:	learn: 2.1518875	total: 203ms	remaining: 33.6s
6:	learn: 2.1326384	total: 240ms	remaining: 34s
7:	learn: 2.1120109	total: 275ms	remaining: 34.1s
8:	learn: 2.0898682	total: 311ms	remaining: 34.2s
9:	learn: 2.0720796	total: 338ms	remaining: 33.4s
10:	learn: 2.0531902	total: 369ms	remaining: 33.2s
11:	learn: 2.0359041	total: 405ms	remaining: 33.3s
12:	learn: 2.0194356	total: 432ms	remaining: 32.8s
13:	learn: 2.0044728	total: 464ms	remaining: 32.7s
14:	learn: 1.9890262	total: 499ms	remaining: 32.8s
15:	learn: 1.9730665	total: 536ms	remaining: 33s
16:	learn: 1.9605564	total: 573ms	remaining: 33.2s
17:	learn: 1.9478802	total: 613ms	remaining: 33.4s
18:	learn: 1.9344954	total: 649ms	remaining: 33.5s
19:	learn: 1.9221579	total: 674ms	remaining

161:	learn: 1.4112195	total: 4.78s	remaining: 24.7s
162:	learn: 1.4098155	total: 4.81s	remaining: 24.7s
163:	learn: 1.4083149	total: 4.85s	remaining: 24.7s
164:	learn: 1.4069944	total: 4.88s	remaining: 24.7s
165:	learn: 1.4054441	total: 4.91s	remaining: 24.6s
166:	learn: 1.4042789	total: 4.93s	remaining: 24.6s
167:	learn: 1.4030268	total: 4.96s	remaining: 24.6s
168:	learn: 1.4016097	total: 4.98s	remaining: 24.5s
169:	learn: 1.3999784	total: 5.01s	remaining: 24.4s
170:	learn: 1.3988366	total: 5.03s	remaining: 24.4s
171:	learn: 1.3976929	total: 5.07s	remaining: 24.4s
172:	learn: 1.3966170	total: 5.1s	remaining: 24.4s
173:	learn: 1.3954534	total: 5.12s	remaining: 24.3s
174:	learn: 1.3944780	total: 5.16s	remaining: 24.3s
175:	learn: 1.3937509	total: 5.18s	remaining: 24.3s
176:	learn: 1.3926500	total: 5.21s	remaining: 24.2s
177:	learn: 1.3910551	total: 5.24s	remaining: 24.2s
178:	learn: 1.3896921	total: 5.27s	remaining: 24.2s
179:	learn: 1.3887362	total: 5.3s	remaining: 24.2s
180:	learn: 1.

320:	learn: 1.2548186	total: 9.51s	remaining: 20.1s
321:	learn: 1.2539210	total: 9.54s	remaining: 20.1s
322:	learn: 1.2529256	total: 9.57s	remaining: 20.1s
323:	learn: 1.2520568	total: 9.61s	remaining: 20s
324:	learn: 1.2517414	total: 9.64s	remaining: 20s
325:	learn: 1.2510164	total: 9.67s	remaining: 20s
326:	learn: 1.2503978	total: 9.69s	remaining: 20s
327:	learn: 1.2490778	total: 9.72s	remaining: 19.9s
328:	learn: 1.2481075	total: 9.75s	remaining: 19.9s
329:	learn: 1.2475022	total: 9.77s	remaining: 19.8s
330:	learn: 1.2466067	total: 9.8s	remaining: 19.8s
331:	learn: 1.2458060	total: 9.83s	remaining: 19.8s
332:	learn: 1.2448138	total: 9.87s	remaining: 19.8s
333:	learn: 1.2440551	total: 9.9s	remaining: 19.7s
334:	learn: 1.2434643	total: 9.93s	remaining: 19.7s
335:	learn: 1.2423139	total: 9.97s	remaining: 19.7s
336:	learn: 1.2414869	total: 10s	remaining: 19.7s
337:	learn: 1.2406856	total: 10s	remaining: 19.7s
338:	learn: 1.2399425	total: 10.1s	remaining: 19.6s
339:	learn: 1.2392261	tota

479:	learn: 1.1418577	total: 14.3s	remaining: 15.5s
480:	learn: 1.1409001	total: 14.3s	remaining: 15.5s
481:	learn: 1.1406606	total: 14.4s	remaining: 15.4s
482:	learn: 1.1395919	total: 14.4s	remaining: 15.4s
483:	learn: 1.1390567	total: 14.4s	remaining: 15.4s
484:	learn: 1.1381752	total: 14.5s	remaining: 15.3s
485:	learn: 1.1375161	total: 14.5s	remaining: 15.3s
486:	learn: 1.1368004	total: 14.5s	remaining: 15.3s
487:	learn: 1.1359753	total: 14.5s	remaining: 15.3s
488:	learn: 1.1353664	total: 14.6s	remaining: 15.2s
489:	learn: 1.1345890	total: 14.6s	remaining: 15.2s
490:	learn: 1.1342765	total: 14.6s	remaining: 15.2s
491:	learn: 1.1339798	total: 14.7s	remaining: 15.1s
492:	learn: 1.1331680	total: 14.7s	remaining: 15.1s
493:	learn: 1.1326531	total: 14.7s	remaining: 15.1s
494:	learn: 1.1314163	total: 14.8s	remaining: 15.1s
495:	learn: 1.1307940	total: 14.8s	remaining: 15s
496:	learn: 1.1303005	total: 14.8s	remaining: 15s
497:	learn: 1.1300635	total: 14.9s	remaining: 15s
498:	learn: 1.1291

638:	learn: 1.0488618	total: 18.9s	remaining: 10.7s
639:	learn: 1.0482636	total: 19s	remaining: 10.7s
640:	learn: 1.0477980	total: 19s	remaining: 10.6s
641:	learn: 1.0473397	total: 19s	remaining: 10.6s
642:	learn: 1.0467051	total: 19.1s	remaining: 10.6s
643:	learn: 1.0463010	total: 19.1s	remaining: 10.6s
644:	learn: 1.0459994	total: 19.1s	remaining: 10.5s
645:	learn: 1.0452319	total: 19.1s	remaining: 10.5s
646:	learn: 1.0448817	total: 19.2s	remaining: 10.5s
647:	learn: 1.0443337	total: 19.2s	remaining: 10.4s
648:	learn: 1.0435224	total: 19.2s	remaining: 10.4s
649:	learn: 1.0428950	total: 19.3s	remaining: 10.4s
650:	learn: 1.0423408	total: 19.3s	remaining: 10.4s
651:	learn: 1.0416572	total: 19.3s	remaining: 10.3s
652:	learn: 1.0408480	total: 19.4s	remaining: 10.3s
653:	learn: 1.0398561	total: 19.4s	remaining: 10.3s
654:	learn: 1.0393532	total: 19.4s	remaining: 10.2s
655:	learn: 1.0388449	total: 19.5s	remaining: 10.2s
656:	learn: 1.0382594	total: 19.5s	remaining: 10.2s
657:	learn: 1.0378

797:	learn: 0.9742546	total: 23.7s	remaining: 6s
798:	learn: 0.9735939	total: 23.7s	remaining: 5.96s
799:	learn: 0.9731134	total: 23.8s	remaining: 5.94s
800:	learn: 0.9724692	total: 23.8s	remaining: 5.91s
801:	learn: 0.9721984	total: 23.8s	remaining: 5.88s
802:	learn: 0.9718654	total: 23.8s	remaining: 5.85s
803:	learn: 0.9713730	total: 23.9s	remaining: 5.82s
804:	learn: 0.9708560	total: 23.9s	remaining: 5.79s
805:	learn: 0.9701646	total: 23.9s	remaining: 5.76s
806:	learn: 0.9695589	total: 24s	remaining: 5.73s
807:	learn: 0.9691413	total: 24s	remaining: 5.7s
808:	learn: 0.9689021	total: 24s	remaining: 5.67s
809:	learn: 0.9687951	total: 24.1s	remaining: 5.64s
810:	learn: 0.9682935	total: 24.1s	remaining: 5.61s
811:	learn: 0.9672133	total: 24.1s	remaining: 5.58s
812:	learn: 0.9667742	total: 24.1s	remaining: 5.55s
813:	learn: 0.9665982	total: 24.2s	remaining: 5.52s
814:	learn: 0.9659734	total: 24.2s	remaining: 5.5s
815:	learn: 0.9654733	total: 24.2s	remaining: 5.47s
816:	learn: 0.9653004	t

956:	learn: 0.9032912	total: 28.3s	remaining: 1.27s
957:	learn: 0.9028901	total: 28.3s	remaining: 1.24s
958:	learn: 0.9023677	total: 28.4s	remaining: 1.21s
959:	learn: 0.9019498	total: 28.4s	remaining: 1.18s
960:	learn: 0.9016745	total: 28.4s	remaining: 1.15s
961:	learn: 0.9013368	total: 28.5s	remaining: 1.13s
962:	learn: 0.9007198	total: 28.5s	remaining: 1.09s
963:	learn: 0.9004184	total: 28.5s	remaining: 1.06s
964:	learn: 0.8998537	total: 28.6s	remaining: 1.04s
965:	learn: 0.8993364	total: 28.6s	remaining: 1.01s
966:	learn: 0.8989798	total: 28.6s	remaining: 977ms
967:	learn: 0.8983064	total: 28.7s	remaining: 948ms
968:	learn: 0.8980213	total: 28.7s	remaining: 919ms
969:	learn: 0.8974991	total: 28.7s	remaining: 889ms
970:	learn: 0.8969255	total: 28.8s	remaining: 860ms
971:	learn: 0.8964832	total: 28.8s	remaining: 830ms
972:	learn: 0.8959174	total: 28.8s	remaining: 800ms
973:	learn: 0.8955142	total: 28.9s	remaining: 770ms
974:	learn: 0.8952569	total: 28.9s	remaining: 741ms
975:	learn: 

## Решение для отправки в систему

In [30]:
import pickle

with open('solution/model.pickle', 'wb') as fout:
    pickle.dump(fire_classifier, fout, protocol=pickle.HIGHEST_PROTOCOL)

## Прогнозирование на новых данных

In [31]:
df_predictions = pandas.DataFrame(
    fire_classifier.predict_proba(X),
    index=df_features.index,
    columns=[
        'fire_{}_prob'.format(class_id)
        for class_id in fire_classifier.classes_
    ],
)
df_predictions.head()

,fire_1_prob,fire_2_prob,fire_3_prob,fire_4_prob,fire_5_prob,fire_6_prob,fire_8_prob,fire_9_prob,fire_10_prob,fire_11_prob
fire_id,,,,,,,,,,
143435,0.000991,0.000767,0.012253,0.000517,0.001748,0.549242,0.114630,0.280387,0.020515,0.018950
155186,0.004872,0.000491,0.134125,0.003067,0.002979,0.190662,0.035799,0.296229,0.203267,0.128509
138374,0.000485,0.003710,0.003922,0.017946,0.001458,0.029627,0.013501,0.179192,0.702873,0.047286
154075,0.026466,0.001038,0.040506,0.010711,0.003378,0.450455,0.056587,0.168754,0.078267,0.163838
138347,0.029806,0.010937,0.012557,0.199349,0.022293,0.066082,0.001847,0.027849,0.538582,0.090698


In [33]:
df_predictions.to_csv('data/sample_predictions_cat.csv', index_label='fire_id')